<a href="https://colab.research.google.com/github/7201krap/PYTORCH_project/blob/main/project_FINAL/sparsity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torchvision import transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
torch.manual_seed(1234)
np.random.seed(1234)

In [ ]:
mnist_trainset = datasets.MNIST(root='./data', train=True, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

mnist_testset  = datasets.MNIST(root='./data', 
                                train=False, 
                                download=True, 
                                transform=transforms.Compose([transforms.ToTensor()]))

train_dataloader = torch.utils.data.DataLoader(mnist_trainset, 
                                               batch_size=50, 
                                               shuffle=True)

test_dataloader  = torch.utils.data.DataLoader(mnist_testset, 
                                               batch_size=50, 
                                               shuffle=False)

print("Training dataset size: ", len(mnist_trainset))
print("Testing dataset size: ",  len(mnist_testset))

In [ ]:
# Define the model 
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.linear_1 = torch.nn.Linear(784, 256)
        self.linear_2 = torch.nn.Linear(256, 10)
        self.sigmoid  = torch.nn.Sigmoid()

    def forward(self, x):
        x = x.reshape(x.size(0), -1)
        x = self.linear_1(x)
        x = self.sigmoid(x)
        pred = self.linear_2(x)

        return pred

In [ ]:
def get_activation(model):    
    def hook(module, input, output):
        model.layer_activations = output
    return hook

In [ ]:
no_epochs = 30
criterion = torch.nn.CrossEntropyLoss()
def sparsity_trainer(optimizer, model):

    # reset the model 
    train_loss = list()
    test_loss  = list()
    test_acc   = list()
    final_spareness = list()

    # define activation list 

    best_test_loss = 1

    for epoch in range(no_epochs):
        total_train_loss = 0
        total_test_loss = 0

        hidden_layer_activation_list = list()

        # training
        # set up training mode 
        model.train()
        for itr, (image, label) in enumerate(train_dataloader):

            image, label = image.to(device), label.to(device)

            optimizer.zero_grad()

            pred = model(image)

            loss = criterion(pred, label)
            total_train_loss += loss.item()

            loss.backward()
            optimizer.step()

        total_train_loss = total_train_loss / (itr + 1)
        train_loss.append(total_train_loss)

        # testing 
        # change to evaluation mode 
        model.eval()
        total = 0
        for itr, (image, label) in enumerate(test_dataloader):
            image, label = image.to(device), label.to(device)

            pred = model(image)

            loss = criterion(pred, label)
            total_test_loss += loss.item()

            # we now need softmax because we are testing.
            pred = torch.nn.functional.softmax(pred, dim=1)
            for i, p in enumerate(pred):
                if label[i] == torch.max(p.data, 0)[1]:
                    total = total + 1

            hidden_layer_activation_list.append(model.layer_activations)

        # this conains activations for all epochs 
        final_spareness.append(hidden_layer_activation_list)

        # caculate accuracy 
        accuracy = total / len(mnist_testset)

        # append accuracy here
        test_acc.append(accuracy)

        # append test loss here 
        total_test_loss = total_test_loss / (itr + 1)
        test_loss.append(total_test_loss)

        # print('\nEpoch: {}/{}, Train Loss: {:.8f}, Test Loss: {:.8f}, Test Accuracy: {:.8f}'.format(epoch + 1, no_epochs, total_train_loss, total_test_loss, accuracy))

        # if total_test_loss < best_test_loss:
        #     best_test_loss = total_test_loss
            # print("Saving the model state dictionary for Epoch: {} with Test loss: {:.8f}".format(epoch + 1, total_test_loss))
            # torch.save(model.state_dict(), "model.dth")

    sparseness_list = list()

    for single_epoch_spareness in final_spareness:
        single_epoch_spareness = torch.stack(single_epoch_spareness)
        layer_activations_list = torch.reshape(single_epoch_spareness, (10000, 256))

        layer_activations_list = torch.abs(layer_activations_list)  # modified 
        num_neurons = layer_activations_list.shape[1]
        population_sparseness = (np.sqrt(num_neurons) - (torch.sum(layer_activations_list, dim=1) / torch.sqrt(torch.sum(layer_activations_list ** 2, dim=1)))) / (np.sqrt(num_neurons) - 1)
        mean_sparseness_per_epoch = torch.mean(population_sparseness)

        sparseness_list.append(mean_sparseness_per_epoch)

    return test_acc, sparseness_list

# AdaDelta

### Rho, weight decay, and learning rate

default: torch.optim.Adadelta(params, lr=1.0, rho=0.9, eps=1e-06, weight_decay=0)


Rho 

In [ ]:
rho_list = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
for i in range(11):
    print(f"Entering {i}-th loop")
    model_adadelta_rho = Model()
    model_adadelta_rho.to(device)
    model_adadelta_rho.sigmoid.register_forward_hook(get_activation(model_adadelta_rho))
    optimizer_adadelta = torch.optim.Adadelta(model_adadelta_rho.parameters(), rho=rho_list[i])
    adadelta_test_acc, adadelta_sparseness_list = sparsity_trainer(optimizer=optimizer_adadelta, model=model_adadelta_rho)
    
    if i == 0:
        f = open("adadelta_rho.txt", "w")
        f.write(str(i)+'\n'+str(adadelta_test_acc)+'\n'+str(adadelta_sparseness_list)+'\n\n')
    else:
        f = open("adadelta_rho.txt", "a")
        f.write(str(i)+'\n'+str(adadelta_test_acc)+'\n'+str(adadelta_sparseness_list)+'\n\n')
f.close()

Weight decay

In [ ]:
weight_decay = np.logspace(-5, 1, 11)

for i in range(11):
    print(f"Entering {i}-th loop")
    model_adadelta_weight_decay = Model()
    model_adadelta_weight_decay.sigmoid.register_forward_hook(get_activation(model_adadelta_weight_decay))
    optimizer_adadelta = torch.optim.Adadelta(model_adadelta_weight_decay.parameters(), weight_decay=weight_decay[i])
    adadelta_test_acc, adadelta_sparseness_list = sparsity_trainer(optimizer=optimizer_adadelta, model=model_adadelta_weight_decay)

    if i == 0:
        f = open("adadelta_weight_decay.txt", "w")
        f.write(str(i)+'\n'+str(adadelta_test_acc)+'\n'+str(adadelta_sparseness_list)+'\n\n')
    else:
        f = open("adadelta_weight_decay.txt", "a")
        f.write(str(i)+'\n'+str(adadelta_test_acc)+'\n'+str(adadelta_sparseness_list)+'\n\n')

f.close()

Learning rate

In [ ]:
learning_rate = np.logspace(-5, 0, 11)

for i in range(11):
    print(f"Entering {i}-th loop")
    model_adadelta_lr = Model()
    model_adadelta_lr.sigmoid.register_forward_hook(get_activation(model_adadelta_lr))
    optimizer_adadelta = torch.optim.Adadelta(model_adadelta_lr.parameters(), lr=learning_rate[i])
    adadelta_test_acc, adadelta_sparseness_list = sparsity_trainer(optimizer=optimizer_adadelta, model=model_adadelta_lr)

    if i == 0:
        f = open("adadelta_learning_rate.txt", "w")
        f.write(str(i)+'\n'+str(adadelta_test_acc)+'\n'+str(adadelta_sparseness_list)+'\n\n')
    else:
        f = open("adadelta_learning_rate.txt", "a")
        f.write(str(i)+'\n'+str(adadelta_test_acc)+'\n'+str(adadelta_sparseness_list)+'\n\n')

f.close()

# AdaGrad

### weight_decay and learning rate

default: torch.optim.Adagrad(params, lr=0.01, lr_decay=0, weight_decay=0, initial_accumulator_value=0, eps=1e-10)

weight_decay

In [ ]:
weight_decay = np.logspace(-5, 1, 11)

for i in range(11):
    print(f"Entering {i}-th loop")
    model_adagrad_weight_decay = Model()
    model_adagrad_weight_decay.sigmoid.register_forward_hook(get_activation(model_adagrad_weight_decay))
    optimizer_adagrad = torch.optim.Adagrad(model_adagrad_weight_decay.parameters(), weight_decay=weight_decay[i])
    adagrad_test_acc, adagrad_sparseness_list = sparsity_trainer(optimizer=optimizer_adagrad, model=model_adagrad_weight_decay)

    if i == 0:
        f = open("adagrad_weight_decay.txt", "w")
        f.write(str(i)+'\n'+str(adagrad_test_acc)+'\n'+str(adagrad_sparseness_list)+'\n\n')
    else:
        f = open("adagrad_weight_decay.txt", "a")
        f.write(str(i)+'\n'+str(adagrad_test_acc)+'\n'+str(adagrad_sparseness_list)+'\n\n')

f.close()

Learning rate

In [ ]:
learning_rate = np.logspace(-5, 0, 11)

for i in range(11):
    print(f"Entering {i}-th loop")
    model_adagrad_lr = Model()
    model_adagrad_lr.sigmoid.register_forward_hook(get_activation(model_adagrad_lr))
    optimizer_adagrad = torch.optim.Adagrad(model_adagrad_lr.parameters(), lr=learning_rate[i])
    adagrad_test_acc, adagrad_sparseness_list = sparsity_trainer(optimizer=optimizer_adagrad, model=model_adagrad_lr)

    if i == 0:
        f = open("adagrad_learning_rate.txt", "w")
        f.write(str(i)+':\n'+str(adagrad_test_acc)+'\n'+str(adagrad_sparseness_list)+'\n\n')
    else:
        f = open("adagrad_learning_rate.txt", "a")
        f.write(str(i)+'\n'+str(adagrad_test_acc)+'\n'+str(adagrad_sparseness_list)+'\n\n')

f.close()

# SGD

### learning rate, weight decay, and momentum

default: torch.optim.SGD(params, lr=<required parameter>, momentum=0, dampening=0, weight_decay=0, nesterov=False)

learning rate

In [ ]:
learning_rate = np.logspace(-5, 0, 11)

for i in range(11):
    print(f"Entering {i}-th loop")
    model_SGD_lr = Model()
    model_SGD_lr.sigmoid.register_forward_hook(get_activation(model_SGD_lr))
    optimizer_SGD = torch.optim.SGD(model_SGD_lr.parameters(), lr=learning_rate[i])
    SGD_test_acc, SGD_sparseness_list = sparsity_trainer(optimizer=optimizer_SGD, model=model_SGD_lr)

    if i == 0:
        f = open("SGD_learning_rate.txt", "w")
        f.write(str(i)+'\n'+str(SGD_test_acc)+'\n'+str(SGD_sparseness_list)+'\n\n')
    else:
        f = open("SGD_learning_rate.txt", "a")
        f.write(str(i)+'\n'+str(SGD_test_acc)+'\n'+str(SGD_sparseness_list)+'\n\n')

f.close()

weight decay

In [ ]:
weight_decay = np.logspace(-5, 1, 11)

for i in range(11):
    print(f"Entering {i}-th loop")
    model_SGD_weight_decay = Model()
    model_SGD_weight_decay.sigmoid.register_forward_hook(get_activation(model_SGD_weight_decay))
    optimizer_SGD = torch.optim.SGD(model_SGD_weight_decay.parameters(), weight_decay=weight_decay[i])
    SGD_test_acc, SGD_sparseness_list = sparsity_trainer(optimizer=optimizer_SGD, model=model_SGD_weight_decay)

    if i == 0:
        f = open("SGD_weight_decay.txt", "w")
        f.write(str(i)+'\n'+str(SGD_test_acc)+'\n'+str(SGD_sparseness_list)+'\n\n')
    else:
        f = open("SGD_weight_decay.txt", "a")
        f.write(str(i)+'\n'+str(SGD_test_acc)+'\n'+str(SGD_sparseness_list)+'\n\n')

f.close()

momentum

In [ ]:
momentum = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

for i in range(11):
    print(f"Entering {i}-th loop")
    model_SGD_lr = Model()
    model_SGD_lr.sigmoid.register_forward_hook(get_activation(model_SGD_lr))
    optimizer_SGD = torch.optim.SGD(model_SGD_lr.parameters(), momentum=momentum[i])
    SGD_test_acc, SGD_sparseness_list = sparsity_trainer(optimizer=optimizer_SGD, model=model_SGD_lr)

    if i == 0:
        f = open("SGD_momentum.txt", "w")
        f.write(str(i)+'\n'+str(SGD_test_acc)+'\n'+str(SGD_sparseness_list)+'\n\n')
    else:
        f = open("SGD_momentum.txt", "a")
        f.write(str(i)+'\n'+str(SGD_test_acc)+'\n'+str(SGD_sparseness_list)+'\n\n')

f.close()

# Adam

### weight decay, learning rate, lhs betas, and rhs betas

default: torch.optim.Adam(params, lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

Weight decay

In [ ]:
weight_decay = np.logspace(-5, 1, 11)

for i in range(11):
    print(f"Entering {i}-th loop")
    model_Adam_weight_decay = Model()
    model_Adam_weight_decay.sigmoid.register_forward_hook(get_activation(model_Adam_weight_decay))
    optimizer_Adam = torch.optim.Adam(model_Adam_weight_decay.parameters(), weight_decay=weight_decay[i])
    Adam_test_acc, Adam_sparseness_list = sparsity_trainer(optimizer=optimizer_Adam, model=model_Adam_weight_decay)

    if i == 0:
        f = open("Adam_weight_decay.txt", "w")
        f.write(str(i)+'\n'+str(Adam_test_acc)+'\n'+str(Adam_sparseness_list)+'\n\n')
    else:
        f = open("Adam_weight_decay.txt", "a")
        f.write(str(i)+'\n'+str(Adam_test_acc)+'\n'+str(Adam_sparseness_list)+'\n\n')

f.close()

Learning rate

In [ ]:
learning_rate = np.logspace(-5, 0, 11)

for i in range(11):
    print(f"Entering {i}-th loop")
    model_Adam_lr = Model()
    model_Adam_lr.sigmoid.register_forward_hook(get_activation(model_Adam_lr))
    optimizer_Adam = torch.optim.Adam(model_Adam_lr.parameters(), lr=learning_rate[i])
    Adam_test_acc, Adam_sparseness_list = sparsity_trainer(optimizer=optimizer_Adam, model=model_Adam_lr)

    if i == 0:
        f = open("Adam_learning_rate.txt", "w")
        f.write(str(i)+'\n'+str(Adam_test_acc)+'\n'+str(Adam_sparseness_list)+'\n\n')
    else:
        f = open("Adam_learning_rate.txt", "a")
        f.write(str(i)+'\n'+str(Adam_test_acc)+'\n'+str(Adam_sparseness_list)+'\n\n')

f.close()

Betas - lhs

In [ ]:
betas_lhs = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

for i in range(11):
    print(f"Entering {i}-th loop")
    model_Adam_lr = Model()
    model_Adam_lr.sigmoid.register_forward_hook(get_activation(model_Adam_lr))
    optimizer_Adam = torch.optim.Adam(model_Adam_lr.parameters(), betas=(betas_lhs[i], 0.999))
    Adam_test_acc, Adam_sparseness_list = sparsity_trainer(optimizer=optimizer_Adam, model=model_Adam_lr)

    if i == 0:
        f = open("Adam_betas_lhs.txt", "w")
        f.write(str(i)+'\n'+str(Adam_test_acc)+'\n'+str(Adam_sparseness_list)+'\n\n')
    else:
        f = open("Adam_betas_lhs.txt", "a")
        f.write(str(i)+'\n'+str(Adam_test_acc)+'\n'+str(Adam_sparseness_list)+'\n\n')

f.close()

Betas - rhs

In [ ]:
betas_rhs = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

for i in range(11):
    print(f"Entering {i}-th loop")
    model_Adam_lr = Model()
    model_Adam_lr.sigmoid.register_forward_hook(get_activation(model_Adam_lr))
    optimizer_Adam = torch.optim.Adam(model_Adam_lr.parameters(), betas=(0.9, betas_rhs[i]))
    Adam_test_acc, Adam_sparseness_list = sparsity_trainer(optimizer=optimizer_Adam, model=model_Adam_lr)

    if i == 0:
        f = open("Adam_betas_rhs.txt", "w")
        f.write(str(i)+'\n'+str(Adam_test_acc)+'\n'+str(Adam_sparseness_list)+'\n\n')
    else:
        f = open("Adam_betas_rhs.txt", "a")
        f.write(str(i)+'\n'+str(Adam_test_acc)+'\n'+str(Adam_sparseness_list)+'\n\n')

f.close()